# Exercise: Weather API

## Aim: Use a Weather API to create and graph NETCDF4 files

### Issues covered:

- Request and get data from a web API endpoint
- Read and retrieve information from a JSON response
- Write and read NETCDF4 files
- Create graphs from NETCDF4 files

## 1. Let's get data from a web API on the internet.

Use this url endpoint `https://api.weather.gov/` as the base.

First, specific to this API, we want to get a station ID and grid co-ordinates. To do so
we will use the `points/{latitude,longitude}` endpoint to get that information.

*Choose the latitude and longitude of your favourite US location (this API is US only and in
latitude North, longitude East)*

To get weather data of a specific location, we use the `gridpoints/{station ID}/{grid co-ordinates}`
endpoint to retrieve so.

In [ ]:
import requests

You can use the requests library to access the webAPI. Fill in the elipses with the `latitude` North and `longitude` East of a location in the US.
If successful, the response code should be 200.

In [ ]:
url = 'https://api.weather.gov/'
latitude = ...
longitude = ...
# Hint: use the requests library to GET from the url: https://api.weather.gov/points/{LAT},{LON}
response = requests.get(f'{url}points/{latitude},{longitude}')
response.status_code

With the requests library, the results from the webAPI can be extracted into a JSON. A JSON is behaves exactly like a dictionary.
Use dictionary indexing to:

- get `gridId`
- get `gridX`
- get `gridY`

In [ ]:
# hint: you can view the JSON on your browser by pasting the URL from the get request.

response = response.json()

gridID = ...
gridX = ...
gridY = ...

With your gridID, gridX, and gridY, use the `gridpoints` endpoint and do a get request, printing the status code.
If everything is working, you should get another 200 status code.

In [ ]:
response = requests.get(f'{url}gridpoints/{gridID}/{gridX},{gridY}')
response.status_code

Can you use the json response data to get the forecast temperature values? Use dictionary indexing to get the `values` from `temperature` in `properties`.

In [ ]:
data = response.json()
forecast = ...

The below code extracts the coordinates of the station you have chosen.

In [ ]:
coords = data['geometry']['coordinates'][0][0]
x = coords[1]
y = coords[0]

## 2. Let's format that data and export as NETCDF4

### Format data

First, format your forecast data to get the datetime and air temperature as separate
lists.

In [ ]:
from datetime import datetime as dt

Loop through your `forecast` values and get the temperatures `value` and dates `validTime` into a list.
`forecast` is a list of dictionaries, where each dictionary is of one time instance.
Fill in the elipses to format the `validTime` string to a python datetime object to the variable date and `value` to the variable temp. They will then be added to the `temps` and `timeseries` list.

In [ ]:
# Use the datetime module to convert the times from the data to a datetime object.
# Hint: look at the validTime string and see how you can turn the string to datetime
#  using strptime, the format of the datetime is in '%Y-%m-%dT%H:%M:%Sz'.

timeseries = []
temps = []

for item in forecast:
    ...
    timeseries.append(date)
    temps.append(temp)

Format the time list you just made and convert to relative time in seconds from the start
the timeseries. The NETCDF using the CF conventions stores time as an offset from a
base time rather than an absolute time.

If you are stuck, take look at the 'Time series' slide in the [`logging data from serial ports`](https://github.com/ncasuk/ncas-isc/blob/master/python/presentations/logging-data-from-serial-ports/LDFSP_Slides.pdf) presentation.

In [ ]:
base_time = timeseries[0]
time_values = []

for t in timeseries:
    ...

time_units = ...

Format the `temps` list you just made to Kelvins. As per CF conventions, the canonical units for Air Temperature is degrees K. Add the Kelvins temperature to the `temp_values` list, just like above.

In [ ]:
temp_values = []

...

### Create NetCDF File


In [ ]:
from netCDF4 import Dataset
import numpy as np

Create the output file (NetCDF dataset):

- Output file to the group workspace.
- Name your file as {Grid ID}-{name}-temps.nc
- The outfile, the location, should be the jasmin shared group workspace directory.

If you need help, have a look at the 'Create the NetCDF dimensions & variables' slide in the [`logging data from serial ports`](https://github.com/ncasuk/ncas-isc/blob/master/python/presentations/logging-data-from-serial-ports/LDFSP_Slides.pdf) presentation.

In [ ]:
dataset = ...

Create the time dimension - with unlimited length

In [ ]:
time_dim = ...

Create the time variable with the following attributes:
- time variables as a numpy float
- fill in the variable with `time_values`
- units as time_units defined earlier
- standard_name as time
- calendar as standard

In [ ]:
time_var = ...
time_var[:] = ...
time_var.units = ...
time_var.standard_name = ...
time_var.calendar = ...

Create the temp variable with the following attributes:
- temp variables as a numpy float
- fill in the variable with `temp_values`
- var_id as temp
- long_name as air temperature (K)
- units as K
- standard_name as air_temperature
- coordinates as the x and y values from the coords variable defined before

In [ ]:
temp_var = ...
temp_var[:] = ...
temp_var.var_id = ...
temp_var.long_name = ...
temp_var.units = ...
temp_var.standard_name = ...
temp_var.coordinates = ...

Set the global attributes as follows:
- who: your name
- where: location of your forecast (grid ID or name)
- when: current datetime as day-month-year hour:minute
- how: forecast data gathered from weather API
- what: air temperature data in K of 7 day forecast

In [ ]:
dataset.who = ...
dataset.where = ...
dataset.when = ...
dataset.how = ...
dataset.what = ...

Save your NetCDF file by closing the dataset.

In [ ]:
dataset.close()

## 3. Import NetCDF4 from Group Workspace

To find all the `.nc` files in a group workspace, we will use the glob module in Python.
Glob let's us find all files matching a pattern, in our case:

`<groupworkspace>/*.nc`

In [ ]:
from glob import glob

Can you use glob to make a list of filepaths of all NetCDF files in the
group workspace?

In [ ]:
filepaths = glob("{...}*temps.nc")

## 4. Graph data from NetCDF
Now that we have a list of netCDF filepaths, we can open them and extract their data.

To start, let us make the plots using matplotlib.

In [ ]:
from netCDF4 import num2date
import matplotlib.pyplot as plt
import  matplotlib.dates as mdates
%matplotlib inline

Create a subplots figure with figure and axis

In [ ]:
fig, ax = ...

Can you set the x-axis locator (ticks) using dates class from matplotlib?
- set the major locator to days.
- set the minor locator to every 6 hours.
- set the x-axis formatter to Day-Month for each day.

In [ ]:
# In the matplotlib.dates module, as mdates, look at the DayLocator and HourLocator.
fmt_day = ...
fmt_six_hours = ...

ax.xaxis.set_major_locator(fmt_day)
ax.xaxis.set_minor_locator(fmt_six_hours)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m'))

Label the axis, `ax`, on the plot:
- label the x-axis as date
- label the y-axis as air temperature / K
- set a title to your plot

In [ ]:
...

Open the netCDF file and extract the `temp` and `time` variables from the file and use
matplotlib `plot_date` function to plot the graph, looping through each file.

- set the label of plot to the coordinate attribute of the temp variable.

Replace the elipses with your plotting, the for loop allows you to plot through all the shared netCDF files in the workspace, where `f` is the datafile and `filepath` is a list of datafiles.

If you need help, look at the 'Plotting data with matplotlib' slide in the [`logging data from serial ports`](https://github.com/ncasuk/ncas-isc/blob/master/python/presentations/logging-data-from-serial-ports/LDFSP_Slides.pdf) presentation.

Plot a line graph using matplotlib: 

- you will need to set the marker to `-` otherwise you will get a scatter graph.
- set the label of the plot to that of the `where` metadata in the netCDF global attributes.

In [ ]:
for f in filepaths:
    ...

Finally, show the plot with a legend, you might want to enable tight layout,
and save the plot to your personal space.

In [ ]:
...